# Introduction
This notebook will model two main things, rigid-body physics, and robotic control systems, both focused around the application of a ping pong playing robot.

## Physics
Rigid-body physics: The physics will pertain to bouncing and rolling of a ping pong ball off of arbitrarily sized, positioned, and rotated, rigid body spheres, rectangular prisms, and cylinders (maybe). I wanted to make a general rigid-body simulation for completeness, but in the end all we really need is just the physics of a ball on a flat table, and an arbitrarily positioned and rotated paddle. The paddle will be made up of either two cylinders, or two rectangular prisms, which would form the handle and the paddle surface.


## Control
Robotic control systems: The first step is to simulate control over the paddle where we can magically set the paddles position and rotation or 6 degrees of freedom (DOF). The second step would be to attach the paddle to the end of a 6-DOF robot arm and control it via inverse kinematics of the robot arm, and simulating motor torques etc.

# Background

## Physics
At its core this notebook is a rigid-body physics simulation. The ball will be affected by gravity, and then bounce off or roll on whichever surfaces it collides with. For bouncing collisions, we will assume that balls mass to be negligible compared to the mass of the paddle/table, so the paddle/table will not budge at all under a collision.

When bouncing, the balls velocity vector will be reflected across the normal of the surface it's colliding with.

When rolling, the ball will have component forces of gravity pushing it in it's rolling state based off the normal of the surface it is rolling on.

## Control
In the first step, the robot's state will be represented as the position and velocity of the ball, followed by the position and rotation of the paddle.

In the second step, the robot's state will be the same, but we will need the velocities and angular velocities of the paddle as well.

The two control modes will be bouncing and rolling, and I will likely implement some sort of sliding mode when the ball is bouncing at a very high frequency. A sliding mode is when the objects state space moves along the edge of the boundary between two different modes, known as the sliding surface.

In the bouncing mode, we can do closed form integration of the balls trajectory, given an initial velocity and position, and then determine where we want to move the paddle to, to intercept it.

In the rolling mode, we will do either PID or state-feedback control to determine how we should position and rotate the paddle to achieve the desired ball position.

# Approach
[Influence Diagram](./influence_diagram.png)

[Simulation Diagram](./simulation_diagram.png)

# Verification
## Physics
I can verify the physics simulation of collisions by doing closed form integration, and compare the analytical results of a given collision trajectory to the simulated results.
For the rolling dynamics I can do a similar thing but that might be harder analytically.
To fully verify the whole physics, I can download a standard physics simulation software and put identical scenarios into my simulation and that software and compare the results.

## Control
This might be out of scope of what I have time to do, but I have access to an actual robot arm, so I might be able to use that for verification. I can make the robot arm parameters match my physical robot arm, and find ideal control gains in simulation, and then use those gains on the physical arm and see if they work.

For this to work we also need closed loop feedback on the position of the ball. I also have access to an XBox360 Kinect (RGBD Camera essentially) which we can use to track the location of the ball. This would obviously add noise and a time delay to the control, so I can try simulating noise and time delay as well!

# Scenarios


# Results (Code for now)

## Imports and Globals

In [8]:
from vpython import *
import numpy as np
import time
G = -9.8
fps = 30
speedMultiplier = 0.5
dt = 0.005 * speedMultiplier
resolutionMultiplier = int(1.0/(dt*fps))
print(resolutionMultiplier)
preSimulateThenPlayback = False

13


## Initial setup

In [2]:
scene = canvas()

mass = 0.0027
radius = 0.02
restitution = 0.8

tableLength = 2.743
tableWidth = 1.524
tableHeight = 0.752
tableSize = vec(2.743, 0.752, 1.524)

origin = sphere(pos=vec(0,0,0), radius=0.1, color=vec(1,0,0))
table = box(pos=tableSize/2, size=tableSize, color=vec(0.3, 0.3, 0.6))
ball = sphere(pos=tableSize/2+vec(0,tableSize.y/2+1,0), radius=radius, color=vec(1,1,1))
ball.vel = vec(0,0,0)

t = 0
endTime = 1
while t < endTime:
    
    gForce = G*mass
    netForce = gForce
    acc = netForce/mass
    ball.vel.y += acc * dt
    ball.pos += ball.vel * dt
    
    if ball.pos.y - ball.radius < table.size.y:
        ball.pos.y = table.size.y + ball.radius
        ball.vel.y *= -restitution
    
    t += dt*resolutionMultiplier
    rate(fps)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Class Definitions

In [3]:
# Returns a rotation matrix R that will rotate vector a to point in the same direction as vector b
def alignVectorsRotationMatrix(a, b):
    b = b / np.linalg.norm(b) # normalize a
    a = a / np.linalg.norm(a) # normalize b
    v = np.cross(a, b)
    # s = np.linalg.norm(v)
    c = np.dot(a, b)
    if np.isclose(c, -1.0):
        return -np.eye(3, dtype=np.float64)

    v1, v2, v3 = v
    h = 1 / (1 + c)

    Vmat = np.array([[0, -v3, v2],
                  [v3, 0, -v1],
                  [-v2, v1, 0]])

    R = np.eye(3, dtype=np.float64) + Vmat + (Vmat.dot(Vmat) * h)
    return R
def axisAngleRotationMatrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    a = cos(theta / 2.0)
    b, c, d = -axis * sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
def yawRotationMatrix(theta):
    return np.array([[cos(theta), 0, sin(theta)],
                     [0, 1, 0],
                     [-sin(theta), 0, cos(theta)]])
    
def pitchRotationMatrix(theta):
    return np.array([[cos(theta), -sin(theta), 0],
                     [sin(theta), cos(theta), 0],
                     [0, 0, 1]])
def rollRotationMatrix(theta):
    return np.array([[1, 0, 0],
                     [0, cos(theta), -sin(theta)],
                     [0, sin(theta), cos(theta)]])

# if m2 is negative, it assumes m2 has infinite mass
# so (m1*v1+m2*v2)/(m1+m2) = m2*v2/m2 = v2 as m2 approaches infinity
def collision(m1, v1, m2, v2, alpha):
    vp = (1+alpha)*((m1*v1+m2*v2)/(m1+m2) if m2 > 0 else v2)
    v1p = vp - alpha*v1
    v2p = vp - alpha*v2
    ## These were for testing
    # kei = 0.5*m1*v1**2 + 0.5*m2*v2**2
    # kef = 0.5*m1*v1p**2 + 0.5*m2*v2p**2
    # foundAlpha = kef / kei
    return v1p, v2p#, kei, kef, foundAlpha

In [4]:
# Abstract
class Object:
    def __init__(self, pos=np.zeros(3)):
        self.pos = np.array(pos, dtype=float)
        self.vel = np.zeros(3)
    
    def show(self):
        self.visual.pos.x = self.pos[0]
        self.visual.pos.y = self.pos[1]
        self.visual.pos.z = self.pos[2]
        
class Sphere(Object):
    def __init__(self, pos=np.zeros(3), radius=1, color=vec(0.4, 0.4, 0.4), make_trail=False):
        super().__init__(pos=pos)
        self.radius = radius
        self.visual = sphere(make_trail = make_trail, pos=vec(0,0,0), radius=radius, color=color)
        self.show()
        
    def move(self, pos=None):
        if np.all(pos != None):
            self.pos = pos
        
    def collideSphere(self, other):
        displacement = other.pos - self.pos
        if np.linalg.norm(displacement) < self.radius + other.radius:
            collisionNormal = displacement/np.linalg.norm(displacement)
            return collisionNormal, collisionNormal*self.radius
        return None, None
        
class Box(Object):
    def __init__(self, pos=np.zeros(3), size=np.ones(3), axis=[1,0,0], angle=0, color=vec(0.4, 0.4, 0.4), showNormals=False):
        super().__init__(pos=pos)
        self.size = np.array(size, dtype=float)
        self.axis = np.array(axis, dtype=float)
        self.angle = 0
        self.visual = box() # make default box for now, it will get redefined in the move
        self.visual.color = color
        self.showNormals = showNormals
        self.arrows = []
        self.move(axis=axis, angle=angle, showNormals=showNormals)
        self.show()
        
    def move(self, pos=None, axis=None, angle=None, showNormals=None):
        if np.all(pos != None):
            self.pos = pos
        if np.all(axis != None):
            self.axis = np.array(axis, dtype=float)
            self.axis /= np.linalg.norm(self.axis)
        if angle != None:
            self.angle = angle
        if showNormals != None:
            self.showNormals = showNormals
            
        self.axisRotation = alignVectorsRotationMatrix(np.array([1,0,0]), self.axis)
        self.angleRotation = axisAngleRotationMatrix(self.axis, self.angle)
        # maybe rework to adjust existing np array instead of creating new one each move
        self.normals = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]], dtype=float)
        for i in range(6):
            self.normals[i] = np.dot(self.axisRotation, self.normals[i])
            self.normals[i] = np.dot(self.angleRotation, self.normals[i])

        
        
        
    def collideSphere(self, other):
        # the dot product is a neat way of finding minimum distance between point and plane
        minDistances = [np.abs(np.dot(other.pos - (self.pos+self.size[int(i/2)]*self.normals[i]*(0 if i%2==1 else 1)), self.normals[i])) for i in range(6)]
        for i in range(6):
            if minDistances[i] < other.radius:
                wallsToCheck = [j for j in range(6) if j != i and j != i+(1 if i%2==0 else -1)]
                outsideWalls = False
                for j in wallsToCheck:
                    if minDistances[j] > self.size[int(j/2)]:
                        # print(str(j) + " | " + str(minDistances))
                        outsideWalls = True
                if not outsideWalls:
                    collisionPoint = other.pos - self.normals[i]*minDistances[i]
                    return self.normals[i], collisionPoint
        return None, None
        
    def update(self):
        pass
            
    def show(self):
        
        self.visual.axis.x = self.axis[0]
        self.visual.axis.y = self.axis[1]
        self.visual.axis.z = self.axis[2]
        
        self.visual.up.x = self.normals[2][0]
        self.visual.up.y = self.normals[2][1]
        self.visual.up.z = self.normals[2][2]
        
        self.visual.size.x = self.size[0]
        self.visual.size.y = self.size[1]
        self.visual.size.z = self.size[2]
        
        # alignedDiagonal = self.size
        alignedDiagonal = np.dot(self.axisRotation, self.size)
        alignedDiagonal = np.dot(self.angleRotation, alignedDiagonal)
        self.pos += alignedDiagonal/2
        super().show()
        if self.showNormals:
            if len(self.arrows)==0:
                for normal in self.normals:
                    self.arrows.append(arrow(pos=vec(self.pos[0],self.pos[1],self.pos[2]), axis=vec(normal[0],normal[1],normal[2])))
            else:
                for i in range(len(self.arrows)):
                    self.arrows[i].axis = vec(self.normals[i][0],self.normals[i][1],self.normals[i][2])
                    self.arrows[i].pos = vec(self.pos[0],self.pos[1],self.pos[2])
                    
        self.pos -= alignedDiagonal/2
        
        
class Ball(Sphere):
    def __init__(self, pos=np.zeros(3), make_trail=False):
        super().__init__(pos=pos, radius=0.02, color=vec(1,1,1), make_trail=make_trail)
        self.restitution = 0.8
        self.mass = 0.0027 # This value is currently irrelevant, because we are assuming all other objects have infinite mass
        
    def update(self, collideables=[]):
        self.vel += np.array([0,1,0])*G*dt
        self.pos += self.vel*dt
        
        
        
        for obj in collideables:
            collisionNormal, collisionPoint = obj.collideSphere(self)
            if type(collisionNormal)!=list:
                collisionNormal = [collisionNormal]
                collisionPoint = [collisionPoint]
            for i in range(len(collisionNormal)):
                if np.all(collisionNormal[i] != None):
                    # First get the new velocity if the collision was perfectly elastic
                    rotMat = alignVectorsRotationMatrix(self.vel*np.array([-1,-1,-1]), collisionNormal[i])
                    self.vel = np.linalg.norm(self.vel)*np.dot(rotMat, collisionNormal[i])
                    
                    # Then find how to adjust it based off of inelasticity
                    # We do this by looking at the magnitude of the velocity projected onto the collision normal
                    projectedVelocityMag = np.dot(self.vel, collisionNormal[i])
                    # And then do a 1d collision equation between both objects projected velocities
                    # Which gives us a new velocity in this 1 dimension.
                    if hasattr(obj, "vel"):
                        otherVel = np.dot(obj.vel, collisionNormal[i])
                    else:
                        otherVel = 0 # Stationary
                    # angular vel in collisions WIP
                    if hasattr(obj, "angularVel"):
                        if np.linalg.norm(obj.angularVel) != 0:
                            # we need to find the instantaneous velocity of the collision point
                            # which is equal to the cross product of the vector from object origin to collision point
                            # and the angular velocity of the object
                            instantaneousRotatingVelocity = np.cross(obj.angularVel, collisionPoint[i])

                            # lastly we project the instantenousRotatingVelocity onto the collision normal
                            rotatingCollisionVelocity = np.dot(instantaneousRotatingVelocity, collisionNormal[i])
                            # and superimpose it with the translational velocity
                            otherVel += rotatingCollisionVelocity
                    # we use negative projectedVelocityMag so the collision function thinks they are colliding
                    newProjectedVelocityMag, _ = collision(self.mass, -projectedVelocityMag, -1, otherVel, self.restitution)
                    # We take the difference in magnitude from before and after the collision
                    collisionAdjustMag = projectedVelocityMag - newProjectedVelocityMag
                    # Then add the difference to the new velocity along the direction of the collision normal
                    collisionAdjust = collisionAdjustMag * collisionNormal[i]
                    if np.dot(self.vel, collisionAdjust) > 0:
                        self.vel -= collisionAdjust
                    else:
                        self.vel += collisionAdjust
                        
                    
                    # Adjust position to snap to collision point
                    self.pos = collisionPoint[i] + self.radius*collisionNormal[i]
        if self.pos[1] < 0:
            self.pos[1] = 0
            self.vel[1] *= -self.restitution
                
        
class Table(Box):
    def __init__(self, pos=np.zeros(3)):
        super().__init__(pos=pos, size=[2.743, 0.752, 1.524], color=vec(0.3, 0.3, 0.6))

class Net(Box):
    def __init__(self, pos=np.zeros(3)):
        super().__init__(pos=pos, size=[0.005, 0.1525, 1.83], color=vec(0.7, 0.7, 0.7))

class PingPongTable():
    def __init__(self, pos=np.zeros(3)):
        self.table = Table(pos=pos)
        self.net = Net(pos=[pos[0]+self.table.size[0]/2,pos[1]+self.table.size[1],pos[2]-0.15])
    def collideSphere(self, other):
        tableNormal, tablePoint = self.table.collideSphere(other)
        netNormal, netPoint = self.net.collideSphere(other)
        return [tableNormal, netNormal], [tablePoint, netPoint]
    def show(self):
        self.table.show()
        self.net.show()
        
class PingPongPaddle():
    def __init__(self, pos=np.zeros(3)):
        handleSize=[0.1, 0.02, 0.03]
        paddleSize=[0.2, 0.02, 0.15]
        # self.handle = Box(pos=[pos[0], pos[1]-handleSize[1]/2, pos[2]-handleSize[2]/2], size=handleSize, showNormals=False)
        # self.paddle = Box(pos=[pos[0]+self.handle.size[0], pos[1]-paddleSize[1]/2, pos[2]-paddleSize[2]/2], size=paddleSize, showNormals=False)
        self.handle = Box(pos=[pos[0], pos[1], pos[2]], size=handleSize, showNormals=False)
        self.paddle = Box(pos=[pos[0]+self.handle.size[0], pos[1], pos[2]-paddleSize[2]/2+self.handle.size[2]/2], size=paddleSize, showNormals=False)
        self.pos = np.array(pos, dtype=float)
        self.axis = self.handle.axis
        self.angle = self.handle.angle
        self.vel = np.zeros(3)
        self.angularVel = np.zeros(3)
    
    # deprecated
    # def update(self):
    #     self.pos += self.vel*dt
    #     self.rotation += self.angularVel*dt
    #     self.axis = np.dot(yawRotationMatrix(self.rotation[0]), np.dot(pitchRotationMatrix(self.rotation[1]), np.array([1,0,0])))
    #     self.angle += self.angularVel[2]*dt
    #     self.move()
    def update(self):
        self.vel = self.vel*0
        self.angularVel = self.angularVel*0
        
    def move(self, pos=None, axis=None, angle=None):
        prevPos = self.pos
        prevAxis = self.axis
        prevAngle = self.angle
        prevUpVector = self.handle.normals[2]
        if np.all(pos != None):
            self.pos = np.array(pos, dtype=float)
            self.vel = (self.pos - prevPos)/dt
        if np.all(axis != None):
            self.axis = np.array(axis, dtype=float)
        if angle != None:
            self.angle = angle
        self.handle.move(pos=self.pos, axis=self.axis, angle=self.angle)
        
        paddleAdjust = np.array([self.handle.size[0], 0, self.handle.size[2]/2-self.paddle.size[2]/2]) # bottom back left corner aligned coordinates
        # paddleAdjust = np.array([self.handle.size[0], -self.paddle.size[1]/2, -self.paddle.size[2]/2]) # principle axis aligned coordinates
        # paddleAdjust = np.array([self.handle.size[0]/2+self.paddle.size[0]/2, 0, 0]) # center points aligned coordinates
        
        # account for rotation
        paddleAdjust = np.dot(self.handle.axisRotation, paddleAdjust)
        paddleAdjust = np.dot(self.handle.angleRotation, paddleAdjust)
        self.paddle.move(pos=self.pos+paddleAdjust, axis=self.axis, angle=self.angle)
        
        # Find angular velocity based on change in axis and up vector
        dAxis = (self.axis - prevAxis) / dt
        dUp = (self.handle.normals[2] - prevUpVector) / dt
        # angularVel = (x X xdot)/(x . x) + x*[(y X ydot)/(y . y) - (x X xdot)/(x . x)]_n / (x-y)_n
        unconstrainedVector = np.cross(prevAxis, dAxis)/np.dot(prevAxis, prevAxis)
        constraintVector = np.cross(prevUpVector, dUp)/np.dot(prevUpVector, prevUpVector) - unconstrainedVector
        constraintScalar = constraintVector[0] / (prevAxis - prevUpVector)[0]
        self.angularVel = unconstrainedVector + prevAxis*constraintScalar
        
        
        # print(f"inst vel: {self.vel} | inst omega: {self.angularVel}")
        
        
        
    def collideSphere(self, other):
        handleNormal, handlePoint = self.handle.collideSphere(other)
        paddleNormal, paddlePoint = self.paddle.collideSphere(other)
        return [handleNormal, paddleNormal], [handlePoint, paddlePoint]
    def show(self):
        self.handle.show()
        self.paddle.show()

## Testing Objects

In [9]:
scene = canvas()
arrow(pos=vec(0,0,0), axis=vec(1,0,0), color=vec(1,0,0))
arrow(pos=vec(0,0,0), axis=vec(0,1,0), color=vec(0,1,0))
arrow(pos=vec(0,0,0), axis=vec(0,0,1), color=vec(0,0,1))

# give ball height 5 for test scenarios
ball = Ball(pos = [0.2, 5, 0.4], make_trail=True)
moveables = [ball]
collideables = []

#### Test physics configurations:
# box1 = Box(pos = [0,0,0], size=[4, 0.4, 4], axis=[-1,1,0], angle=0, showNormals=True)
# box2 = Box(pos = [0,0,0], size=[4, 0.4, 4], axis=[1,1,0*-0.05], angle=2*3.14159/180, showNormals=False, color=vec(0.6, 0.4, 0.4))
# collideables.append(box1)
# collideables.append(box2)

# ball.vel[0]=5.0
# ball.vel[2]=0.07
# table = PingPongTable()
# collideables.append(table)
# collideables.append(Box(pos=[-1,0,0], size=[1,6,3]))
# collideables.append(Box(pos=[table.table.size[0],0,0], size=[1,6,3]))

paddle = PingPongPaddle(pos=[0,1,0.4])
# paddle.vel = np.array([0, 0.1, 0])
# paddle.angularVel = np.array([0, 0.3, 0.3])
collideables.append(paddle)
moveables.append(paddle)

# PADDLE MOVING UP AND ROTATING TEST
# collision info with moving paddle (paddle pos, axis, angle, then ball pos, vel)
# [0.         1.37613477 0.4       ]
# [1.      0.08515 0.08515]
# 0.2554499999999885
# [0.2        1.43680575 0.4       ]
# [-1.5445572   6.89103511  3.78784728]

# ball outgoing velocity with stationary paddle
# [-1.49416243  5.59639537  3.59652008]
# paddle.move(pos = [0.0, 1.37613477, 0.4], axis = [1.0, 0.08515, 0.08515], angle = 0.2554499999999885)

# PADDLE JUST ROTATING TEST TOWARDS BALL
# [0.  1.  0.4]
# [1.     0.1784 0.1784]
# 0.5351999999999743
# [0.2        1.09031205 0.4       ]
# [-3.23014701  2.68774666  6.66705852]

# [0.  1.  0.4]
# [1.     0.1784 0.1784]
# 0.5351999999999743
# [0.2        1.09031205 0.4       ]
# [-3.13376291  2.34641491  6.46812071]
# paddle.move(axis = [1.0, 0.1784, 0.1784], angle = 0.5351999999999743)

# PADDLE JUST ROTATING TEST AWAY FROM BALL
# [0.  1.  0.4]
# [1.      0.22084 0.22084]
# -0.5374799999999604
# [0.2        1.06356697 0.4       ]
# [-1.03605302  3.27137072 -7.3627115 ]

# [0.  1.  0.4]
# [1.      0.22084 0.22084]
# -0.5374799999999604
# [0.2        1.06356697 0.4       ]
# [-0.98428974  2.66912227 -6.99485572]

# paddle.move(axis = [1.0, 0.22084, 0.22084], angle = -0.5374799999999604)





moveablesStates = []
for _ in moveables:
    moveablesStates.append([])
t = 0
simLengthSeconds = 0.5
simLengthTicks = simLengthSeconds/speedMultiplier
startTime = time.time()
while t < simLengthTicks:
    
    for _ in range(resolutionMultiplier):
        # startTime = time.time()
        ball.update(collideables=collideables)
        paddle.update()
        dumbVal = 0.4 - t / simLengthTicks
        # paddle.move(pos=[0, 1+sin(t)*0.5, 0.4], axis=[1,dumbVal, dumbVal], angle=3*t/simLengthTicks)
        # move paddle towards ball
        # paddle.move(axis=[1,dumbVal, dumbVal], angle= 3*t / simLengthTicks)
        # away from
        t += dt
        # endTime = time.time()
        paddle.move(axis=[1,dumbVal, dumbVal], angle= -3*t / simLengthTicks)
        
    
    
    if preSimulateThenPlayback:
        for i in range(len(moveables)):
            moveablesStates[i].append(np.copy(moveables[i].pos))
    if not preSimulateThenPlayback:
        for moveable in moveables:
            moveable.show()
        rate(fps)
endTime = time.time()

if preSimulateThenPlayback:
    print(f"Simulation took: {endTime - startTime} seconds")
    for t in range(len(moveablesStates[0])):
        for i in range(len(moveablesStates)):
            moveables[i].move(pos=moveablesStates[i][t])
            moveables[i].show()
        rate(fps)
else:
    print(f"Simulation was expected to take: {t} seconds and it took: {endTime - startTime} seconds")


<IPython.core.display.Javascript object>

Simulation was expected to take: 1.0074999999999896 seconds and it took: 1.867882490158081 seconds


### Odd random tests

In [6]:
# Find b given a,c and a X b = c (infinite solutions)
# but because our rotation has multiple of these equations
# we should be able to find a solution

# xdot = (xnew - x)/dt
# we have xdot = w X x
# gives us     w = (x X xdot)/(x . x) + t*x
# we also have w = (y X ydot)/(y . y) + t*y
# we also have w = (z X zdot)/(z . z) + t*z # but we dont need this one
# we get: (x X xdot)/(x . x) + t*x = (y X ydot)/(y . y) + t*y
# yields: t*(x-y) = (y X ydot)/(y . y) - (x X xdot)/(x . x)
# looking elementwise:
# t*(x-y)_n = [(y X ydot)/(y . y) - (x X xdot)/(x . x)]_n
# t = [(y X ydot)/(y . y) - (x X xdot)/(x . x)]_n / (x-y)_n
# now solving for w
# w = (x X xdot)/(x . x) + x*[(y X ydot)/(y . y) - (x X xdot)/(x . x)]_n / (x-y)_n

scene = canvas()
a = np.array([1, 0, 0])
c = np.array([1, 1, 0])

arrow(pos=vec(0,0,0), axis=vec(a[0], a[1], a[2]), color=vec(1,0,0))
arrow(pos=vec(0,0,0), axis=vec(c[0], c[1], c[2]), color=vec(0,1,0))

t=0
b = np.cross(c, a)/np.dot(a, a) + t*a
bArrow = arrow(pos=vec(0,0,0), axis=vec(b[0], b[1], b[2]), color=vec(0,0,1))
while t < 1:
    b = np.cross(c, a)/np.dot(a, a) + t*a
    bArrow.axis = vec(b[0], b[1], b[2])
    rate(10)
    t += 0.1

<IPython.core.display.Javascript object>

In [7]:
# [-3.07017752  3.1438587   6.73900347]

# [0.  1.  0.4]
# [1.     0.1784 0.1784]
# 0.5351999999999743
# [0.2        1.09031205 0.4       ]
# [-3.13376291  2.34641491  6.46812071]

print(np.linalg.norm(np.array([-3.07017752, 3.1438587, 6.73900347])))
print(np.linalg.norm(np.array([-3.13376291, 2.34641491, 6.46812071])))

8.04512307541352
7.560603046390736


# Conclusions and Future Work


# References

[Phet collision sim](https://phet.colorado.edu/sims/html/collision-lab/latest/collision-lab_all.html)

[Physics Libre Text on CoM Collisions](https://phys.libretexts.org/Bookshelves/University_Physics/Mechanics_and_Relativity_(Idema)/04%3A_Momentum/4.08%3A_Elastic_Collisions_in_the_COM_Frame)

[Youtube video on rigid body dynamics](https://www.youtube.com/watch?v=4r_EvmPKOvY&list=PLwMZtAEBQ8ZywWPf6twbspmYzGg0Fr2DJ&index=5)